In [ ]:
import random
from google.colab import drive

def generate_valid_ean13():
    """Δημιουργεί έγκυρο EAN-13 barcode με σωστό check digit"""
    # Πρώτα 12 τυχαία ψηφία
    ean = [random.randint(0, 9) for _ in range(12)]

    # Υπολογισμός check digit (σωστός αλγόριθμος)
    sum_even = sum(ean[i] for i in range(1, 12, 2))  # Άρτια positions (2nd, 4th,...)
    sum_odd = sum(ean[i] for i in range(0, 12, 2))   # Περιττά positions (1st, 3rd,...)
    total = sum_odd * 3 + sum_even
    check_digit = (10 - (total % 10)) % 10

    # Επιστροφή ως string 13 ψηφίων
    return ''.join(map(str, ean)) + str(check_digit)

def replace_ean_codes(input_file, output_file):
    """Αντικαθιστά ΟΛΑ τα EAN codes στο SQL αρχείο"""
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if 'INSERT INTO `ticket`' in line:
                # Χωρίζουμε τη γραμμή για να βρούμε το EANCode
                parts = line.split('VALUES (')
                if len(parts) == 2:
                    values = parts[1].split(',')
                    if len(values) > 3:  # Το EANCode είναι στη 4η θέση (index 3)
                        # Αντικατάσταση με νέο EAN-13
                        values[3] = ' ' + generate_valid_ean13()  # Προσοχή στο κενό πριν
                        new_line = parts[0] + 'VALUES (' + ','.join(values)
                        outfile.write(new_line)
                        continue
            outfile.write(line)

def main():
    # Σύνδεση με Google Drive
    drive.mount('/content/drive')

    # Διαδρομές αρχείων
    input_path = input("ticketeanfix.sql").strip()
    output_path = 'fixed3' + input_path.split('/')[-1]

    full_input = f"/content/drive/MyDrive/ticketeanfix.sql"
    full_output = f"/content/drive/MyDrive/{output_path}"

    try:
        replace_ean_codes(full_input, full_output)
        print(f"Επιτυχία! Όλα τα EAN codes αντικαταστάθηκαν. Το αρχείο αποθηκεύτηκε ως: {output_path}")
    except Exception as e:
        print(f"Σφάλμα: {str(e)}")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ticketeanfix.sql
Επιτυχία! Όλα τα EAN codes αντικαταστάθηκαν. Το αρχείο αποθηκεύτηκε ως: fixed3
